In [1]:
## Basic stuff
%load_ext autoreload
%autoreload
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
display(HTML("""<style>div.output_area{max-height:10000px;overflow:scroll;}</style>"""))

## Python Version
import sys
print("Python: {0}".format(sys.version))

from ioUtils import saveJoblib, loadJoblib, saveFile, getFile
import urllib
from urllib.parse import quote

from discogsBase import discogs
from discogsUtils import discogsUtils
from collection import collections
from artist import artist
from timeUtils import clock, elapsed
from fsUtils import moveFile, setFile, setDir, setSubDir, isFile, isDir, mkDir
from fileUtils import getFileBasics, getBasename
from artists import artists
from albums import albums
from time import sleep

import datetime as dt
start = dt.datetime.now()
print("Notebook Last Run Initiated: "+str(start))

Python: 3.7.3 (default, Mar 27 2019, 16:54:48) 
[Clang 4.0.1 (tags/RELEASE_401/final)]
Notebook Last Run Initiated: 2020-01-15 19:26:33.561036


In [2]:
%load_ext autoreload
%autoreload
disc   = discogs()
arts   = artists(disc)
albs   = albums(disc)
dutils = discogsUtils()
col    = collections(disc)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Saved Discog Directory /Volumes/Music/Discog is Available
Local Discog Directory /Users/tgadfort/Music/Discog is Available
/Volumes/Music/Discog/collections exists
/Volumes/Music/Discog/artists exists
/Volumes/Music/Discog/albums exists
/Volumes/Music/Discog/collections-db exists
/Volumes/Music/Discog/artists-db exists
/Volumes/Music/Discog/albums-db exists
/Volumes/Music/Discog/artists-db/metadata exists
/Volumes/Music/Discog/albums-db/metadata exists
/Volumes/Music/Discog/diagnostic exists
/Volumes/Music/Discog/db exists


# Parse Artists DB

In [3]:
#arts.parseArtistModValFiles(65, force=True)

Parsing Artist Files For ModVal 65
Forcing Reloads of ModVal=65
  Processing 6704 files.
	Processed 0/6704 files.
	Processed 500/6704 files.
	Processed 1000/6704 files.
	Processed 1500/6704 files.
	Processed 2000/6704 files.
	Processed 2500/6704 files.
	Processed 3000/6704 files.
	Processed 3500/6704 files.
	Processed 4000/6704 files.
	Processed 4500/6704 files.
	Processed 5000/6704 files.
	Processed 5500/6704 files.
	Processed 6000/6704 files.
	Processed 6500/6704 files.
Saving 6704 new artist IDs to /Volumes/Music/Discog/artists-db/65-DB.p
  --> This file is 12.3MB.
Saving 6704 new artist IDs name data to /Volumes/Music/Discog/artists-db/metadata/65-Metadata.p
  --> This file is 391.5kB.
Saving 6704 new artist IDs media data to /Volumes/Music/Discog/artists-db/metadata/65-MediaMetadata.p
  --> This file is 4.0MB.


6704

# Missing Artists From Albums

In [ ]:
%load_ext autoreload
%autoreload
from albums import albums
from discogsBase import discogs

disc  = discogs()
albs  = albums(disc)
toget = albs.findMissingArtistsFromAlbums()

In [ ]:
from time import sleep
togetmap  = toget['togetmap']
refstoget = toget['toget']
for ia,item in enumerate(refstoget.most_common()):
    artistID, cnt = item[0], item[1]
    ref      = togetmap[artistID]
    url      = arts.getArtistURL(ref)
    savename = arts.getArtistSavename(artistID)
    if not isFile(savename):
        try:
            print('\n{0}/{1}'.format(ia,len(refstoget)))
            retval = arts.downloadArtistURL(url, savename, debug=True)
            sleep(2)
        except:
            continue
            
    #print(item)
#len(togetmap)

# Albums 

In [ ]:
#arts.parseArtistModValFiles(98, force=True)
arts.parseArtistModValFiles(49, force=True)
#for modVal in range(92,100):
#    parseArtists(modVal)

# Download Artist Albums

In [ ]:
arts.searchDiscogForArtist("Royce Lovett")
arts.searchDiscogForArtist("Melanie Sparks")

In [ ]:
%load_ext autoreload
%autoreload
from albums import albums
disc = discogs()
albs = albums(disc)
albs.downloadAlbumsFromArtists(maxAlbums=1, mediaTypes=None, rand=True)

# Parallel Parse Artists

In [ ]:
%load_ext autoreload
%autoreload

from artist import artist
from artists import artists
arts = artists(disc)

def parseArtists(modVal, force=False, doExtra=False):
    arts.parseArtistModValFiles(modVal, force=force)
    #arts.parseArtistModValExtraFiles(modVal)


def parseArtistsParallel(nProc=3, force=False):
    pool = Pool(processes=nProcs)
    result = pool.map_async(parseArtists, range(100)) 
    start, cmt = clock("Processing {0} modVals using {1} processes".format(modVals, nProcs))
    while not result.ready():
        if force is True:
            time.sleep(10)
        else:
            time.sleep(1)
    print("")
    
    elapsed(start, cmt)
    return result.get()

start,cmt=clock("Parsing All Artists")
start2,cmt2=clock("Parallel Artist Parsing")
#parseArtistsParallel()
elapsed(start2, cmt2)
#arts.buildMetadata(force=False, doAlbums=False)
elapsed(start, cmt)

# Missing Artists From Albums

In [ ]:
%load_ext autoreload
%autoreload
from albums import albums
from discogsBase import discogs

disc  = discogs()
albs  = albums(disc)
toget = albs.findMissingArtistsFromAlbums()

In [ ]:
from time import sleep
togetmap  = toget['togetmap']
refstoget = toget['toget']
for ia,item in enumerate(refstoget.most_common()):
    artistID, cnt = item[0], item[1]
    ref      = togetmap[artistID]
    url      = arts.getArtistURL(ref)
    savename = arts.getArtistSavename(artistID)
    if not isFile(savename):
        try:
            print('\n{0}/{1}'.format(ia,len(refstoget)))
            retval = arts.downloadArtistURL(url, savename, debug=True)
        except:
            continue
            
        sleep(2)
    #print(item)
#len(togetmap)

# Albums Of Matched People

In [ ]:
data = getFile("/Volumes/Music/MusicDB/toget.p")

In [ ]:
%load_ext autoreload
%autoreload
from albums import albums
disc = discogs()
discdf = disc.getMasterDiscogsDB()
print(discdf.shape)
discdf = discdf[~discdf['Name'].isna()]
print(discdf.shape)

artists = list(discdf["Artist"])

_ = clock("Last Run")

In [ ]:
def getMusicData(key, artist):
    retval = discdf[discdf[key] == artist]
    if retval.shape[0] > 0:
        return retval
    else:
        return None
    
toget={}
known={}

for ia,artistName in enumerate(data):
    #artistID = artistName
    artistID = None
    if artistID is None:
        retval = getMusicData('Name', artistName)
        if retval is not None:
            artistID = retval.index.values[0]
    modValue  = dutils.getDiscIDHashMod(discID=artistID, modval=disc.getMaxModVal())
    if toget.get(modValue) is None:
        toget[modValue] = {}
    toget[modValue][artistID] = True
        
    if ia % 100 == 0:
        print(ia,'/',len(data))

In [ ]:
list(reversed(list(toget.keys())))

In [ ]:
for modVal in toget.keys():
    artistIDs = toget[modVal]
    print(modVal,'\t',len(artistIDs))
    dbdata = disc.getArtistsDBModValData(modVal)
    for artistID in artistIDs:
        print(' ',artistID)
        artistData = dbdata[artistID]
        albs.downloadAlbumFromArtistData(artistID=artistID, artistData=artistData, mediaTypes=None)

In [ ]:

        
1/0

ref = retval['Ref']
print(artistName,ref)
refval = ref.values[0]
artistAlbums = retval["Albums"]
print(artistAlbums)
continue
if artistAlbums is not None:
    for mediaType,mediaData in artistAlbums.items():
        print(mediaType,mediaData)
        for albumID,albumURL in mediaData.items():
            savename = self.getAlbumSavename(artistID, albumID)                    
            if not isFile(savename):
                toGet[albumURL] = savename
            else:
                known[albumURL] = savename

        baseURL = self.disc.discogURL
        url = urllib.parse.urljoin(baseURL, quote(albumURL))
        if knownAlbums.get(url) is True:
            print("\t  Previously downloaded.")
            continue
        retval = self.downloadAlbumURLData(url, savename, artistID)                

        retval = self.downloadAlbumURLData(url, savename, artistID)
    1/0




In [ ]:
ref.values[0]

In [ ]:
albums

# Extra Media

In [ ]:
def findErrs(modVal, show=True):
    print("Finding Errors For {0}".format(modVal))
    dbdata = disc.getArtistsDBModValData(modVal)

    dels = []
    for artistID,artistInfo in dbdata.items():
        ref    = artistInfo.url.url

        nMedia = sum([len(x) for x in artistInfo.media.media.values()])
        counts = 0 #artistInfo.mediaCounts.counts["Releases"]["All"]
        pages  = artistInfo.pages
        if pages.tot is None:
            print(artistID,'\t',ref,'\t',nMedia,'\t',pages.get(),'\t',counts)
            if show:
                continue
                
            dels.append(artistID)
            
            artistID = dutils.getArtistID(ref)
            url = arts.getArtistURL(ref)
            savename = arts.getArtistSavename(artistID)
            arts.downloadArtistURL(url=url, savename=savename, debug=True, force=True)


        elif (pages.tot > pages.ppp or nMedia % 25 == 0) and pages.pages > 1:
            print(artistID,'\t',ref,'\t',nMedia,'\t',pages.get(),'\t',counts)
            if show:
                continue
                
            dels.append(artistID)
            
            artistID = dutils.getArtistID(ref)
            url = arts.getArtistURL(ref)
            savename = arts.getArtistSavename(artistID)
            arts.downloadArtistURL(url=url, savename=savename, debug=True, force=True)

    #arts.rmIDFromDB(dels)
    if show:
        return

    saveVal = False
    for ID in dels:
        try:
            del dbdata[ID]
            print("Deleted {0}".format(ID))
            saveVal = True
        except:
            print("Not there...")
            
    dbname = disc.getArtistsDBModValFilename(modVal)
    if saveVal:
        print("Saving {0}".format(dbname))
        saveFile(idata=dbdata, ifile=dbname)
    else:
        print("No reason to save {0}".format(dbname))

    arts.parseArtistModValFiles(modVal, force=False)
    arts.assertDBModValExtraData(modVal)
    arts.parseArtistModValExtraFiles(modVal)

In [ ]:
#findErrs(98, show=False)

# Credits

In [ ]:

artistIDs = disc.getArtistIDToNameData()
from collections import Counter
toget    = Counter()
togetmap = {}

#for modVal in ['NAN'] + list(range(disc.getMaxModVal())):
db = disc.getAlbumsDBModValData(2)



# Search For Artists

In [ ]:
%load_ext autoreload
%autoreload
disc   = discogs()
arts   = artists(disc)

In [ ]:
artist = "Suzanne Haik Vantoura"
artist = "A Spanner In The Works"
artist = "L-Gee"
artist = "Zakary Thaks"
artist = "Apollo Four Forty"
artist = "Beatsnblends"
artist = "Bipolar Outing"
artist = "4th Of July"

arts.searchDiscogForArtist(artist)

# My Music

In [ ]:
artistData = getFile(ifile="/Volumes/Music/MusicDB/artistData.p")
albumData  = getFile(ifile="/Volumes/Music/MusicDB/albumData.p")



In [ ]:
artistData["Camouflage (West German ''Target'' Pressing)"]

# Extra Artists

In [ ]:
retval = albs.findMissingArtistsFromAlbums()
togetmap = retval["togetmap"]
toget    = retval["toget"]

In [ ]:
if False:
    from time import sleep
    for item in toget.most_common():
        artistID, cnt = item[0], item[1]
        ref      = togetmap[artistID]
        url      = arts.getArtistURL(ref)
        savename = arts.getArtistSavename(artistID)
        if not isFile(savename):
            try:
                arts.downloadArtistURL(url, savename, debug=True)
            except:
                continue
            sleep(2)
        #print(item)
    #len(togetmap)

# Collections By Artist

In [ ]:
from collections import Counter
db = Counter(disc.getCollectionRefCountsData())
coreIDs = disc.getArtistIDAlbumIDs()

In [ ]:
tmp = getFile('/Volumes/Music/Discog/db/AlbumIDToArtistID.p')
artIDtoAlbID = {}
for k,v in tmp.items():
    for v2 in v:
        if artIDtoAlbID.get(v2) is None:
            artIDtoAlbID[v2] = {}
        artIDtoAlbID[v2][k] = True
for k in artIDtoAlbID.keys():
    artIDtoAlbID[k] = list(artIDtoAlbID[k].keys())

In [ ]:
for item in db.most_common(500):
    ref,cnt  = item[0],item[1]
    artistID = dutils.getArtistID(ref)
    modVal   = str(int(artistID[-2:]))
    
    coreAlbums = coreIDs.get(artistID)
    #if len(coreAlbums) % 25 == 0:
    print("{0: <50}{1: <5}{2: <10}{3: <5}{4: <5}".format(ref,modVal,artistID,len(coreAlbums),len(artIDtoAlbID[artistID])))
    continue
    if False:
        print(artistID,'\t',ref)
        arts.rmIDFromDB(artistID)
        url = arts.getArtistURL(ref)
        savename = arts.getArtistSavename(artistID)
        arts.downloadArtistURL(url=url, savename=savename)
        sleep(2)
    #print(artistID,item,len(coreAlbums))

In [ ]:
albs.downloadAlbumsFromArtists(maxAlbums=1)

In [ ]:
dbArt = disc.getArtistIDToNameData()
dbCol = disc.getCollectionIDToRefData()
artistDBDir = disc.getArtistsDBDir()

In [ ]:

refs = [dbCol.get('489428')]
for ref in refs:
    if ref is None:
        continue
    print("REF --> {0}".format(ref))
    href = ref.replace("https://www.discogs.com/artist/", "")
    artistID = dutils.getArtistID(ref)
    modValue = dutils.getDiscIDHashMod(discID=artistID, modval=disc.getMaxModVal())

    if False:
        dbname  = setFile(artistDBDir, "{0}-DB.p".format(modValue))     
        print("Loading {0}".format(dbname))
        dbdata  = getFile(dbname)

        try:
            del dbdata[artistID]
            print("Deleted {0}".format(artistID))
        except:
            print("Not there...")

        print("Saving {0}".format(dbname))
        saveFile(idata=dbdata, ifile=dbname)

    url = arts.getArtistURL(ref)
    savename = arts.getArtistSavename(artistID)
    arts.downloadArtistURL(url=url, savename=savename, parse=True, force=True, debug=True)
    sleep(1)

In [ ]:
vals = ["1166600", "1324400", "1340600", "15800", "1626700", "1691500", "2006300", "211200", "23500", "2360300", "2701000", "29300", "40000", "4300900", "437700", "444000", "4731500", "563200", "6400", "72800", "90800"]

In [ ]:
modVal=0

artistDBDir = disc.getArtistsDBDir()
dbname  = setFile(artistDBDir, "{0}-DB.p".format(modVal))     
dbdata  = getFile(dbname)
print("Found {0} artist IDs in {1}".format(len(dbdata), dbname))

for val in vals:
    del dbdata[val]

savename = setFile(artistDBDir, "{0}-DB.p".format(modVal))     
print("Saving {0} new artist IDs to {1}".format(len(dbdata), savename))
saveJoblib(data=dbdata, filename=savename, compress=True)


In [ ]:
from collections import Counter
toget = Counter()
togetmap = {}
for modVal in ['NAN'] + list(range(disc.getMaxModVal())):
    db = disc.getAlbumsDBModValData(modVal)
    for artistID,artistData in db.items():
        for albumID,albumData in artistData.items():
            artists = albumData.artist
            if artists is None:
                continue
            for artist in artists:
                artistInfo = artist[0]
                ID  = artistInfo.ID
                ref = artistInfo.url
                if ID is not None and artistIDs.get(ID) is None:
                    toget[ID] += 1
                    togetmap[ID] = ref
                    
    print(modVal,'\t',len(toget))

In [ ]:
vals="""Downloading: https://www.discogs.com/artist/302000-Ballard-MacDonald?sort=year%2Casc&limit=500&page=2
Downloading: https://www.discogs.com/artist/75910-Stephen-W-Tayler?sort=year%2Casc&limit=500&page=2
Downloading: https://www.discogs.com/artist/11415-Bob-Clearmountain?sort=year%2Casc&limit=500&page=2
Downloading: https://www.discogs.com/artist/834222-Paul-Myers-2?sort=year%2Casc&limit=500&page=2
Downloading: https://www.discogs.com/artist/674125-Ram%C3%83%C2%B3n-Arcusa?sort=year%2Casc&limit=500&page=2
Downloading: https://www.discogs.com/artist/253127-Fredrik-Nordstr%C3%83%C2%B6m?sort=year%2Casc&limit=500&page=2
Downloading: https://www.discogs.com/artist/449529-Denny-Purcell?sort=year%2Casc&limit=500&page=2
Downloading: https://www.discogs.com/artist/273544-Angus-Young?sort=year%2Casc&limit=500&page=2
Downloading: https://www.discogs.com/artist/1641558-Jacques-Fournier?sort=year%2Casc&limit=500&page=2
Downloading: https://www.discogs.com/artist/252962-Alfred-Lion?sort=year%2Casc&limit=500&page=2
Downloading: https://www.discogs.com/artist/244167-Brendan-OBrien?sort=year%2Casc&limit=500&page=2
Downloading: https://www.discogs.com/artist/27468-Bill-Bottrell?sort=year%2Casc&limit=500&page=2
Downloading: https://www.discogs.com/artist/382969-Fred-Kevorkian?sort=year%2Casc&limit=500&page=2
Downloading: https://www.discogs.com/artist/142073-Bernard-Edwards-Nile-Rodgers?sort=year%2Casc&limit=500&page=2
Downloading: https://www.discogs.com/artist/255681-Bill-Szymczyk?sort=year%2Casc&limit=500&page=2
Downloading: https://www.discogs.com/artist/332586-Ralph-Bass?sort=year%2Casc&limit=500&page=2
Downloading: https://www.discogs.com/artist/180298-Tom-Dowd?sort=year%2Casc&limit=500&page=2"""

# Bad Artists

In [ ]:
5442814                           /artist/5442814-Shad-Music
1876816                    /artist/1876816-Mattia-Travaglini
517916                            /artist/517916-Sam-Feldman
19522             /artist/19522-Psychic-TV-Genesis-P-Orridge
3187022                  /artist/3187022-WOI-PH-Sleep-MSM-RE
2835023                         /artist/2835023-Dilraaj-Kaur
4807923                               /artist/4807923-Mr-Rod
2120424                      /artist/2120424-Aaron-Collins-4
277424     /artist/277424-The-Austrian-State-Radio-Orchestra
3316725                        /artist/3316725-Pavel-Krasnov
5470627                       /artist/5470627-Petar-Bratanov
5470628                      /artist/5470628-Kamen-Grahovski
5470629                     /artist/5470629-Nikolay-Berberov
2864703    /artist/2864703-Scottish-Chamber-Orchestra-Chorus
3191803                       /artist/3191803-Mark-T-Johnson
2168730                         /artist/2168730-Fulvio-Locci
298530                          /artist/298530-Stuart-Eltham
3357530              /artist/3357530-Matt-Saltlicker-Lambeth
4955231                              /artist/4955231-LGBProd
3498232        /artist/3498232-Nora-Olearius-Matthias-Keller
2825035                  /artist/2825035-Anastasia-Romanenko
306940                           /artist/306940-Dale-Steward
3463348                           /artist/3463348-Tino-Isgro
4927849                              /artist/4927849-PH-Boer
3851405                             /artist/3851405-Noggin-4
3962251                        /artist/3962251-Dj-Fat-Franco
4884051                         /artist/4884051-John-Modenos
1046052                       /artist/1046052-Manuel-Soler-2
2605855                       /artist/2605855-Dino-Paredes-2
5166458                         /artist/5166458-Elwood-Doran
3961406               /artist/3961406-SIE-Speed-Is-Essential
2117360                      /artist/2117360-Ray-M-Flores-IV
540270                             /artist/540270-L-Ferguson
2181671                       /artist/2181671-Ciaran-Sweeney
2971073                       /artist/2971073-Mike-Douglas-5
3465573                     /artist/3465573-Jeff-Christopher
458475                         /artist/458475-Sebastian-Hoff
5402008                        /artist/5402008-Yahman-Celvai
190580                            /artist/190580-Nuno-Rebelo
2426380             /artist/2426380-Davide-BrutalDave-Billia
4645580                      /artist/4645580-Ormur-Ãlafsson
5128482                          /artist/5128482-Ras-Tom-EYE
1880086          /artist/1880086-Frankfurt-Baroque-Orchestra
63888                 /artist/63888-Rhymes-Rhythm-Blues-Band
4184291                         /artist/4184291-Fuyuki-Tenge
3705398      /artist/3705398-ÐÐµÐ¾ÑÐ³Ð¸Ð¹-ÐÐ¶Ð°Ð³Ð°ÑÐ¾Ð²
3743398                             /artist/3743398-Mia-Berg

In [ ]:
%load_ext autoreload
%autoreload
disc   = discogs()
art    = artist()
arts   = artists(disc)

In [ ]:
atc = art.getData('/Volumes/Music/Discog/artists/98/3743398.p')

In [ ]:

artistDBDir = disc.getArtistsDBDir()

/artist/67340-Yaki-Da

urls = vals.split('\n')
for url in urls:
    ref = url.split('?')[0]


    href = ref.replace("https://www.discogs.com/artist/", "")
    artistID = dutils.getArtistID(url)
    modValue = dutils.getDiscIDHashMod(discID=artistID, modval=disc.getMaxModVal())

    if False:
        dbname  = setFile(artistDBDir, "{0}-DB.p".format(modValue))     
        print("Loading {0}".format(dbname))
        dbdata  = getFile(dbname)

        try:
            del dbdata[artistID]
            print("Deleted {0}".format(artistID))
        except:
            print("Not there...")

        print("Saving {0}".format(dbname))
        saveFile(idata=dbdata, ifile=dbname)
    
    url = ref.replace("Downloading: ", "")
    savename = arts.getArtistSavename(artistID)
    arts.downloadArtistURL(url=url, savename=savename, force=True, debug=True)
    sleep(1)

In [ ]:
for modVal in range(1,100):
    try:
        arts.assertDBModValData(modVal)
    except:
        pass

In [ ]:
for modVal in [1]:
    start, cmt = clock("Processing {0}".format(modVal))
    arts.parseArtistModValFiles(modVal, force=True)
    elapsed(start, cmt)

In [ ]:
data = getFile("/Volumes/Music/Discog/artists-db/1-DB.p")

In [ ]:
artistIDtoRefData = disc.getArtistIDToRefData()

In [ ]:
for modVal in range(20,60):
    start, cmt = clock("Processing {0}".format(modVal))
    arts.parseArtistModValFiles(modVal, force=True)
    elapsed(start, cmt)

In [ ]:
x = str(artistData.url.url)

In [ ]:
x.Contains("x")

In [ ]:
errs=getFile("errs.p")
print("Found {0} err files".format(len(errs)))
from collections import Counter
n = 0
for item in Counter(refCounts).most_common():
    n += 1
    ref      = item[0]
    cnts     = item[1]
    if errs.get(ref):
        continue
    discID   = arts.discogsUtils.getArtistID(ref)
    url      = arts.getArtistRef(ref)
    savename = arts.getArtistSavename(discID)
    if isFile(savename):
        continue
    try:
        arts.downloadArtistURL(url, savename)
        sleep(1)
    except:
        print("Adding {0} to errs".format(ref))
        errs[ref] = True
        if len(errs) % 10 == 0:
            saveFile(idata=errs, ifile="errs.p", debug=True)
        pass
    
    print("----------> {0} \t {1}".format(cnts,n)
    print("")



In [ ]:
import time
from multiprocessing import Pool
 
artistDir = disc.getArtistsDir()
maxModVal = disc.getMaxModVal()

artistDBDir = disc.getArtistsDBDir()        

def f(a_list):
    out = None
    for n in a_list:
        dbname = setFile(artistDBDir, "{0}-DB.p".format(i))            
        dbdata = getFile(dbname, version=3)
        return dbdata
        if out is None:
            out = dbdata
        else:
            out.update(dbdata)
 
    return out
 
def f_amp(a_list):
    chunks = [a_list[i::5] for i in range(5)]
 
    print(chunks)
    pool = Pool(processes=5)
 
    result = pool.map_async(f, chunks)
    print(result)
 
    while not result.ready():
        print("Running...")
        time.sleep(5)
 
    return result.get()
    #return sum(result.get())
    
f_amp(files[:5])

In [ ]:
f_amp(range(100))

In [ ]:
from searchUtils import findExt

artistDir = disc.getArtistsDir()
maxModVal = disc.getMaxModVal()

artistDBDir = disc.getArtistsDBDir()        

totalSaves = 0
for i in range(maxModVal):
    dirVal = setDir(artistDir, str(i))
    files  = findExt(dirVal, ext='.p')
    break

    #dbname = setFile(artistDBDir, "{0}-DB.p".format(i))            
    #dbdata = getFile(dbname, version=3)

In [ ]:
len(files)

In [ ]:
chunks = [files[:10][i::5] for i in range(5)]

In [ ]:
[len(x) for x in chunks]

In [ ]:
%load_ext autoreload
%autoreload
disc = discogs()
arts = artists(disc)
dutils = discogsUtils()
col = collections(disc)

In [ ]:
from time import sleep
stracks = getFile("soundtracks.p")
from numpy import ceil
for cnt,style in stracks.items():
    print(cnt,'\t',style)
    maxmp = int(ceil(int(cnt.replace(',', ""))/250))
    col.downloadCollection(maxPages=min([15,maxmp]), style=["Soundtrack",style], master=True) #, fmat=fmat)

In [ ]:
import time
from multiprocessing import Pool
 
artistDir = disc.getArtistsDir()
maxModVal = disc.getMaxModVal()

artistDBDir = disc.getArtistsDBDir()   

In [ ]:
def f(a_list, dbdata):
    out = []
    for ifile in a_list:
        discID = getBaseFilename(ifile)
        if dbdata.get(discID) is None:
            out.append(ifile)
            
    return out
 
def f_amp(a_list):
    chunks = [a_list[i::5] for i in range(5)]
 
    print(chunks)
    pool = Pool(processes=5)
    
    
    dbname = setFile(artistDBDir, "{0}-DB.p".format(67))            
    dbdata = getFile(dbname, version=3)
 
    result = pool.map_async(f, chunks)
    print(result)
 
    while not result.ready():
        print("Running...")
        time.sleep(5)
 
    return sum(result.get())
    #return sum(result.get())
    
f_amp(files[:5])

In [ ]:
a1 = "Duke Ellington"
a2 = "Coleman Hawkins"

In [ ]:
disc     = discogs()
artistDB = disc.getArtistVariationNameToIDsData()

In [ ]:
artistDB[a1]

In [ ]:
artistDB[a2]

In [ ]:
artistIDToName = disc.getArtistIDToNameData()
artistIDToRef = disc.getArtistIDToRefData()

In [ ]:
artistIDToRef['284747']

In [ ]:
artistIDToName['284747-1']

In [ ]:
from artists import artists
from artist import artist

In [ ]:
art = artist()
arts = artists(disc)

In [ ]:
ifile = '/Volumes/Music/Discog/artists/47/284747.p'
info = art.getData(ifile)

In [ ]:
len(info['Media']["Albums"])

In [ ]:
discID   = '284747'
href     = artistIDToRef[discID]
url      = arts.getArtistRef(href)
savename = arts.getArtistSavename(discID)
print(url)
url="{0}/?sort=year%2Casc&limit=500".format(url)
arts.downloadArtistURL(url, savename, force=True)

In [ ]:
html = str(getFile(ifile))
type(html)
html.find("1,221")

In [ ]:
len(html)

In [ ]:
info.keys()

for key in info.keys():
    print("\n\n\n{0}".format(key))
    val = info[key]
    if isinstance(val, dict):
        for key2,val2 in val.items():
            if isinstance(val2, list):
                print("\t{0}".format(key2))
                for key3 in val2:
                    print("\t\t{0}".format(key3))
            elif isinstance(val2, dict):
                print("\t{0}".format(key2))
                for key3,val3 in val2.items():
                    print("\t\t{0}".format(key3))
                    print("\t\t\t{0}".format(val3))
            else:
                print("\t{0}:\t{1}".format(key2,val2))
    else:
        print(info[key])

In [ ]:
a = artist()

In [ ]:
x = a.getData(ifile)

In [ ]:
bsdata = getHTML(ifile)

In [ ]:
x['']

In [ ]:
apc.err

In [ ]:
apc.__dict__

In [ ]:
bsdata = getHTML(ifile)

In [ ]:
pages

In [ ]:
tot

In [ ]:
int(ceil(tot/ppp))

In [ ]:
artistDir = disc.getArtistsDir()
maxModVal = disc.getMaxModVal()

artistDBDir = disc.getArtistsDBDir()

artistIDToRef = disc.getArtistIDToRefData()

In [ ]:

%load_ext autoreload
%autoreload

from artist import artist
from searchUtils import findExt
from fileUtils import getBaseFilename
artistInfo = artist()
toget={}
extras={}
totalSaves = 0
for i in range(19,maxModVal):
    dirVal = setDir(artistDir, str(i))
    files  = findExt(dirVal, ext='.p')

    saveIt = 0
    nf = len(files)
    for j,ifile in enumerate(files):
        artistID = getBaseFilename(ifile)
        if toget.get(artistID) is not None:
            continue
            
        info     = artistInfo.getData(ifile)

        ID       = info.ID.ID
            
        if artistID != ID:
            extras[ifile] = info
            continue
            
        ref = info.url.url
            
        pages  = info.pages
        if pages.redo is True:
            toget[artistID] = ref
            if len(toget) % 25 == 0:
                print(j,'/',nf,'\t',i,'\t',len(toget),'\t',ID,'\t',ref)
                
    saveFile(idata=toget, ifile="toget3.p")

In [ ]:

%load_ext autoreload
%autoreload

from artist import artist
from artists import artists

from searchUtils import findExt
from fileUtils import getBaseFilename
artistInfo = artist()

redo={}
more={}
extras={}
totalSaves = 0
for i in range(1, 8):
    dirVal = setDir(artistDir, str(i))
    files  = findExt(dirVal, ext='.p')

    saveIt = 0
    nf = len(files)
    for j,ifile in enumerate(files):
        artistID = getBaseFilename(ifile)
        if redo.get(artistID) is not None:
            continue
        if more.get(artistID) is not None:
            continue
            
        info     = artistInfo.getData(ifile)

        ID       = info.ID.ID
            
        if artistID != ID:
            extras[ifile] = info
            continue
            
        ref    = info.url.url
        pages  = info.pages
        if pages.redo is True:
            redo[artistID] = ref
            print(j,'/',nf,'\t',i,'\t',len(toget),'\t',ID,'\t',ref,'\t',pages.get())
            
        if pages.more is True:
            more[artistID] = ref
            print(j,'/',nf,'\t',i,'\t',len(toget),'\t',ID,'\t',ref,'\t',pages.get())

                
    #saveFile(idata=toget, ifile="toget.p")

In [ ]:
%load_ext autoreload
%autoreload

from artist import artist
from artists import artists

for artistID,artistRef in more.items():
    ifile = arts.getArtistSavename(artistID)
    info  = art.getData(ifile)
    pages = info.pages
    
    npages = pages.pages

    print(ifile,'\t',artistRef,'\t',pages.get())

    for p in range(2, npages+1):
        url      = arts.getArtistURL(artistRef, p)
        savename = arts.getArtistSavename(artistID, p)
        if not isFile(savename):
            arts.downloadArtistURL(url=url, savename=savename, force=True, debug=True)

In [ ]:
tmp = {k: redo[k] for k in ['1028302', '1323902', '1411102', '200402', '2159702', '228202']}
tmp

In [ ]:
redo = tmp.copy()
redo

In [ ]:
redo

In [ ]:
import urllib
from urllib.parse import quote
from webUtils import getHTML

def getHeaders():
    user_agent = 'Mozilla/5.0 (Windows; U; Windows NT 5.1; en-US; rv:1.9.0.7) Gecko/2009021910 Firefox/3.0.7'
    headers={'User-Agent':user_agent,} 
    return headers
    
def searchDiscogs(query, requireUS=False, artist=False, master=False, release=False):

    print("\n\n===================== Searching For \"{0}\" =====================".format(query))
    baseURL = disc.discogSearchURL

    url = urllib.parse.urljoin(baseURL, "{0}{1}{2}".format("?q=", quote(query), "&limit=250"))
    if release:
        url = "{0}&type=release".format(url)
    elif master:
        url = "{0}&type=master".format(url)
    elif artist:
        url = "{0}&type=artist".format(url)
        
    if requireUS:
        url = "{0}&country_exact=US".format(url)
    
    sleep(1)

    print("Downloading: {0}".format(url))

    headers = getHeaders()
    request=urllib.request.Request(url,None,headers) #The assembled request
    response = urllib.request.urlopen(request)
    data   = response.read() # The data u need
    bsdata = getHTML(data)
    return bsdata
        

In [ ]:
%load_ext autoreload
%autoreload


def getRefName(ref):
    try:
        href   = ref.attrs.get('href')
        name   = ref.text.strip()
    except:
        href   = None
        name   = None
        
    return href,name


from artist import artist
from artists import artists
from album import album
from albums import albums
albs   = albums(disc)
arts   = artists(disc)
dutils = discogsUtils()
from discogsBase import discogs
disc = discogs()
bsdata = searchDiscogs("Duke Ellington with the Ron Collier Orchestra Collages", requireUS=True, release=True)

h4s = bsdata.findAll("h4")
h5s = bsdata.findAll("h5")

albumData  = [x for x in h4s if len(x.findAll('a')) > 0]
albumData  = [x.findAll('a') for x in albumData if not x.find('a').attrs['href'].startswith("/sell/")]
artistData = [x.findAll("a") for x in h5s if len(x.findAll('span')) > 0]
downloadData = zip(albumData,artistData)

baseURL = disc.discogURL
for (albumDatas,artistDatas) in downloadData:
    for albumData in albumDatas:
        albumRef,albumName = getRefName(albumData)
        try:
            albumID  = str(int(albumRef.split("/")[-1]))
        except:
            print("Could not get album ID from {0}".format(albumRef))
            albumID  = None
        url = urllib.parse.urljoin(baseURL, albumRef)
        print(url)
        retval = albs.downloadAlbumURL(url=url)


        for artistData in artistDatas:
            artistRef,artistName = getRefName(artistData)
            artistID = dutils.getArtistID(artistRef)
            albs.saveDownloadedAlbumURL(albumdata=retval, albumID=albumID, artistID=artistID, debug=True)

In [ ]:
%load_ext autoreload
%autoreload

from artist import artist
from artists import artists
from album import album
from albums import albums

albs = albums(disc)

albs.getData(retval)

In [ ]:
from copy import deepcopy
ab = deepcopy(retval)

In [ ]:
from ioUtils import getFile
from fsUtils import isFile
from fsUtils import removeFile
from fileUtils import getsize
from webUtils import getHTML, isBS4
from strUtils import fixName
from math import ceil, floor
from collections import OrderedDict

from discogsBase import discogs
from discogsUtils import discogsUtils

class albumCodeClass:
    def __init__(self, code=None, err=None):
        self.code = code
        self.err  = err
        
    def get(self):
        return self.__dict__
    
            
class albumURLClass:
    def __init__(self, url=None, err=None):
        self.url = url
        self.err = err
        
    def get(self):
        return self.__dict__
        
        
class albumNameClass:
    def __init__(self, name=None, err=None):
        self.name = name
        self.err  = err
        
    def get(self):
        return self.__dict__
    

class albumBasicClass:
    def __init__(self, artist=None, album=None, err=None):
        self.artist  = artist
        self.album   = album
        self.err     = err
        
    def get(self):
        return self.__dict__

    
class albumProfileClass:
    def __init__(self, label=None, aformat=None, country=None, released=None, genre=None, style=None, err=None):
        self.label    = label
        self.aformat  = aformat
        self.country  = country
        self.released = released
        self.genre    = genre
        self.style    = style
        self.err      = err
        
    def get(self):
        return self.__dict__
    

class albumTrackClass:
    def __init__(self):
        self.tracks = []
        self.err    = None
        
    def add(self, track):
        self.tracks.append(track)
        if track.err is not None:
            self.err = "Track {0}".format(len(self.tracks))
        
    def get(self):
        return self.__dict__
    

class albumTrackDataClass:
    def __init__(self, name=None, length=None, position=None, err=None):
        self.name      = length
        self.length    = length
        self.position  = position
        self.err       = err
        
    def get(self):
        return self.__dict__
    

class albumVersionClass:
    def __init__(self, text=None, nums=None, err=None):
        self.text = text
        self.nums = nums
        self.err  = err
        
    def get(self):
        return self.__dict__
    

class albumCreditClass:
    def __init__(self):
        self.credit = {}
        self.err    = None
        
    def add(self, role, ref, err=None):
        self.credit[role] = ref
        if err is not None:
            self.err = "Role: {0}".format(role)
        
    def get(self):
        return self.__dict__
    

class albumURLInfo:
    def __init__(self, name=None, url=None, ID=None, err=None):
        self.name = name
        self.url  = url
        self.ID   = ID
        self.err  = err
        
    def get(self):
        return self.__dict__
        
        
class albumDataClass:
    def __init__(self, artist=None, album=None, profile=None, url=None, code=None, tracks=None, versions=None, credits=None, err=None):
        self.artist   = artist
        self.album    = album
        self.profile  = profile
        self.url      = url
        self.code     = code
        self.tracks   = tracks
        self.versions = versions
        self.credits  = credits
        self.err      = err
                
    def get(self):
        return self.__dict__




class album(discogs):
    def __init__(self):
        self.bsdata    = None
        self.filename  = None
        self.debug     = False
        self.dutils    = discogsUtils()

    def stripName(self, name):
        if name == None:
            return None
        
        if isinstance(name,str):
            if len(name) > 2:
                if name[:2] == "\n ":
                    name = name[2:].strip()
                if name[-1] == "\n":
                    name = name[:-1].strip()
        return name

        
        
    def getData(self, inputdata):
        bsdata = None
        if isinstance(inputdata, bytes):
            try:
                bsdata = getHTML(str(inputdata))
            except:
                raise ValueError("Not sure about btypes input: {0}".format(inputdata))
        elif isinstance(inputdata, str):
            if isFile(inputdata):
                try:
                    self.filename = inputdata
                    bsdata = getHTML(getFile(inputdata))
                except:
                    try:
                        bsdata = getHTML(getFile(inputdata, version=2))
                    except:
                        raise ValueError("Cannot read artist file: {0}".format(inputdata))
                        
                if self.filename is not None:
                    if getsize(self.filename) < 1000:
                        raise ValueError("Not a real file: {0}".format(self.filename))
            else:
                try:
                    bsdata = getHTML(inputdata)
                except:
                    raise ValueError("Not sure about string input: {0}".format(inputdata))
        elif isBS4(inputdata):
            bsdata = inputdata
        else:
            raise ValueError("Not sure about input type: {0}".format(type(inputdata)))

        self.bsdata = bsdata
        print(self.filename)
        
        return self.parse()
    

    def getNamesAndURLs(self, content):
        data = []
        for ref in content.findAll("a"):
            url    = ref.attrs['href']
            name   = fixName(ref.text)
            try:
                ID = self.dutils.getArtistID(url)
            except:
                ID = None
                            
            data.append(albumURLInfo(name=name, url=url, ID=ID))
        return data


    def cleanContent(self, key, content, debug = False):
        for i,item in enumerate(content):
            name   = self.stripName(item[0])
            if debug: 
                if name != item[0]:
                    print("\t",key,'\t\t',item[0],' -> ',name)
            url    = item[1]
            discID = item[2]
            content[i] = [name,url,discID]

        return content


    def getAlbumBasics(self):
        info = {"Err": None}

        result = self.bsdata.find("div", {"class": "profile"})
        if result:
            spans = result.findAll("span")
            for span in spans:
                attrs = span.attrs
                if "title" in attrs.keys():
                    artist = self.getNamesAndURLs(span)
                    if info.get("Artist") == None:
                        info["Artist"] = []
                    info["Artist"].append(artist)
                    if self.debug:
                        print("Found Artist:",artist)
                else:
                    album = fixName(span.text)
                    album = album.strip()
                    album = album.replace("\n", "")
                    info["Album"] = album
                    if self.debug:
                        print("Found Album:",info["Album"])

        if info.get("Artist") == None:
            info["Err"] = "No Artist"

        ## Fix artist list (if needed)
        if info.get("Artist") is not None:
            artists = []
            for artist in info["Artist"]:
                if isinstance(artist, list):
                    if isinstance(artist[0], list):
                        artists.append(artist[0])
                    else:
                        artists.append(artist)
                else:
                    artists.append(artist)
            info["Artist"] = artists
        else:
            info["Artist"] = None

        
        abc = albumBasicClass(artist=info["Artist"], album=info["Album"], err=info["Err"])
        return abc


    def getAlbumProfile(self):
        result = self.bsdata.find("div", {"class": "profile"})
        data   = {"Err": None}
        if result:
            heads = result.findAll("div", {"class": "head"})
            heads = [x.text for x in heads]
            heads = [x.replace(":","") for x in heads]

            content = result.findAll("div", {"class": "content"})
            if len(heads) != len(content):
                data["Err"] = "Mismatch in head/content"

            for i in range(min(len(heads),len(content))):
                reftest = content[i].find("a")
                if reftest:
                    content[i] = self.getNamesAndURLs(content[i])
                    if heads[i] == "Country" or heads[i] == "Released":
                        content[i][0].name = self.stripName(content[i][0].name)
                else:
                    content[i] = artistURLInfo(name=content[i].text.strip())
                data[heads[i]] = content[i]

            apc = albumProfileClass(label=data.get("Label"), aformat=data.get("Format"),
                                    country=data.get("Country"), released=data.get("Released"),
                                    genre=data.get("Genre"), style=data.get("Style"),
                                    err=data.get("Err"))
        else:
            apc = albumProfileClass(err="No Profile")                
                
        return apc



    def getAlbumTracks(self):
        atc = albumTrackClass()
        
        result = self.bsdata.find("table", {"class": "playlist"})
        if result is None:
            atc.err = "No Tracks"
        else:
            trackLines = result.findAll("tr", {"class": "track"})
            if len(trackLines) == 0:
                trackLines = result.findAll("tr", {"class": "tracklist_track"})
                
            for tr in trackLines:
                # position
                attrs = tr.attrs
                position = None
                name = None
                url  = None
                duration = None
                if attrs:
                    position = attrs.get("data-track-position")

                # name
                td = tr.find("td", {"class": "track"})
                if td:
                    span = td.find("span", {"class": "tracklist_track_title"})
                    if span:
                        name = fixName(span.text)

                    meta = td.find("meta")
                    if meta:
                        url = meta.attrs.get("content")

                # duration
                td = tr.find("td", {"class": "tracklist_track_duration"})
                if td:
                    span = td.find("span")
                    if span:
                        duration = span.text

                if position is None:
                    position = len(tracks)+1
                    
                if name is None:
                    err = "No Name"
                elif duration is None:
                    err = "No Duration"
                else:
                    err = None

                track = albumTrackDataClass(name=name, length=duration, position=position, err=err)
                atc.add(track)

                if self.debug:
                    print(position,name,duration)

        return atc



    def getAlbumURL(self):
        url = None
        
        # 1st Try
        result = self.bsdata.find("link", {"rel": "canonical"})
        if result is not None:
            url = result.attrs["href"]
            url = url.replace("https://www.discogs.com", "")        
            if self.debug:
                print("Found URL using rel: canoical -->",url)

        # 2nd Try
        if url is None:
            result = self.bsdata.find("link", {"hreflang": "en"})
            if result is not None:
                url = result.attrs["href"]
                url = url.replace("https://www.discogs.com", "")
                if self.debug:
                    print("Found URL using hreflang: en -->",url)

        if url is not None:
            auc = albumURLClass(url=url)
        else:
            auc = albumURLClass(err="No URL")            
                    
        return auc



    def getAlbumCode(self, url):
        err  = None
        code = None
        
        try:
            code = url.split('/')[-1]
            code = str(int(code))
        except:
            err  = "NaN"

        if code == None:
            if url.find("Various?anv=") != -1:
                code = -1
                err = "Various"

        acc = albumCodeClass(code=code, err=err)
        return acc



    def getAlbumVersions(self):
        err  = None
        text = None
        nums = None
        h3s  = [x for x in alb.bsdata.findAll("h3") if x.attrs.get('data-for') == ".m_versions"]
        if len(h3s) == 1:
            try:
                text = removeTag(h3s[0], tag='a').text.strip()
            except:
                text = None
                err  = "LinkErr"

            if text is not None:
                try:
                    nums = text[text.find("(")+1:text.find(")")].split(" of ")
                except:
                    nums = None
                    err  = "NumErr"

                if nums is not None:
                    try:
                        nums = [int(x) for x in nums]
                    except:
                        nums = None
                        err  = "NumNotInt"
        else:
            err = "NoVersions"

        avc = albumVersionClass(text=text, nums=nums, err=err)
        return avc



    def getAlbumCredits(self):
        acc = albumCreditClass()
        
        result = self.bsdata.find("div", {"class": "credits"})
        if result:
            for li in result.findAll("li"):
                role = li.find("span", {"class": "role"})
                if role:
                    role = role.text
                href = self.getNamesAndURLs(li)
                acc.add(role, href)
        else:
            acc.err = "NoCredits"

        return acc



    def parse(self, debug = False):
        basics = self.getAlbumBasics()
        artist = basics.artist
        album  = basics.album
        
        profile     = self.getAlbumProfile()
        url         = self.getAlbumURL()
        code        = self.getAlbumCode(url.url)
        tracks      = self.getAlbumTracks()
        versions    = self.getAlbumVersions()
        credits     = self.getAlbumCredits()

        
        err = [basics.err, profile.err, url.err, code.err, tracks.err, versions.err, credits.err]
        
        adc = albumDataClass(artist=artist, album=album, profile=profile, url=url, code=code, tracks=tracks, versions=versions, credits=credits)
        
        return adc

In [ ]:
alb = album()
ad  = alb.getData('/Volumes/Music/Discog/albums/57/145257/4900039.p')

In [ ]:
ad.artist[1][0].get()

In [ ]:

        for h3 in self.bsdata.findAll("h3"):
            attrs = h3.attrs
            if attrs.get("data-for"):
                if attrs["data-for"] == ".m_versions":
                    for val in h3.strings:
                        if val.find("Versions") != -1:
                            versions = val.replace("Versions", "")
                            versions = versions.replace("(", "")
                            versions = versions.replace(")", "")
                            versions = versions.strip()
                            if self.debug:
                                print("Found versions:",versions)
                            return versions

        if self.debug:

In [ ]:
soundtracks = sorted(list(set(tmp["Album"].to_list())))

term = "Walt Disney's "
soundtracks = [x.replace(term, "") if x.startswith(term) else x for x in soundtracks]
term="Music From The Motion Picture Sound Track "
soundtracks = [x.replace(term, "") if x.startswith(term) else x for x in soundtracks]
term="Music From The Motion Picture Soundtrack "
soundtracks = [x.replace(term, "") if x.startswith(term) else x for x in soundtracks]
term="Music From The Original Motion Picture Soundtrack "
soundtracks = [x.replace(term, "") if x.startswith(term) else x for x in soundtracks]
term="Music From The Original Sound Track "
soundtracks = [x.replace(term, "") if x.startswith(term) else x for x in soundtracks]
term="Music From The Soundtrack Of "
soundtracks = [x.replace(term, "") if x.startswith(term) else x for x in soundtracks]

terms = []
terms.append(" (Original Cast Sound Track)")
terms.append(" (Original Cast Soundtrack)")
terms.append(" (Music From The Original Motion Picture Sound Track)")
terms.append(" (Music From The Soundtrack)")
terms.append(" (Music from the soundtrack of)")
terms.append(" (Original Motion Picture Soundtrack)")
terms.append(" (Original Sound Track Recording)")
terms.append(" (Original Soundtrack)")
terms.append(" (Original Sound Track Recording)")
terms.append(" (The Original Sound Track Recording)")
terms.append(" (Original Soundtrack)")
terms.append(" (Original Soundtrack Recording)")
terms.append(" (Music From The Motion Picture Sound Track)")
terms.append(" (Music From The Motion Picture Soundtrack)")
terms.append(":  Music From The Original Motion Picture Soundtrack")
terms.append(" - Music From The Original Motion Picture Soundtrack")
terms.append(" (Music From The Motion Picture)")
terms.append(" (An Original Soundtrack Recording)")
terms.append("  (An Original Soundtrack Recording)")
terms.append(" Original Motion Picture Soundtrack")

terms+=['(Original Television Soundtrack Recording)',
 '(Motion Picture Soundtrack)',
 '(The Original Motion Picture Soundtrack)',
 "(The Original Motion Picture Soundtrack)",
 '(Music From The Original Motion Picture Soundtrack)',
 '(Original Motion Picture Soundtrack Album)',
 "(Music From The Original Motion Picture Soundtrack)",
 "(Original TV Soundtrack)",
 "(From The Motion Picture Soundtrack)",
 '(Soundtrack)',
 '(Limited Event Series Soundtrack)',
 '(Music From The Original Motion Picture Soundtrack)',
 '(Music Composed For The Original Motion Picture Soundtrack)',
 '(Music From The Original Motion Picture Soundtrack)',
 '(The Original Soundtrack Recording)',
 '(original Soundtrack From The Movie)',
 '(The Original Movie Soundtrack)',
 '(The Original Soundtrack From The Motion Picture)',
 '(Music From The Original Motion Picture Soundtrack)',
 '(Music From The Original Soundtrack)',
 '(From The Original Motion Picture Soundtrack)'
 ' - Original Soundtrack Recording',
 ' (An Original Sound Track Recording)',
 ' - Music from the soundtrack of the Motion Picture',
 ' Original Soundtrack',
 ' Original Cast Sound Track Album',
 ' - Original Cast Sound Track Album',
 ' (From The Original Sound Track)',
 ' - Music From The Original Motion Picture Sound Track',
 ' (The Original Motion Picture Sound Track)',
 ' - Original Sound Track Recording',
 ': Original Cast Soundtrack',
 ':  Original Cast Soundtrack',
 ' (Original Cast Sound Track Album)']
for term in terms:
    soundtracks = [x.replace(term, "").strip() if x.endswith(term) else x for x in soundtracks]

print(len(soundtracks))
soundtracks = set(soundtracks)
print(len(soundtracks))
soundtracks